# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data = pd.read_csv("output_data/cities.csv")
data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Maumere,-8.62,122.21,79.86,71,83,3.80,ID,06/21/20
1,Tautira,-17.73,-149.15,84.20,62,75,5.82,PF,06/21/20
2,Rikitea,-23.12,-134.97,70.79,68,86,18.01,PF,06/21/20
3,Sur,22.57,59.53,85.53,68,58,9.26,OM,06/21/20
4,Ancud,-41.87,-73.82,48.00,100,90,16.11,CL,06/21/20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Let us configure gmaps

gmaps.configure(api_key=g_key)
locations = data[["Lat", "Lng"]]
humidity = data["Humidity"]

In [4]:
# Let's plot the heatmap

fig = gmaps.figure(center=(20, 0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3.5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Following are the conditions that are to be met in order to have the perfect vacation!
# 1. A max temperature lower than 80 degrees but higher than 70.
# 2. Wind speed less than 10 mph.
# 3. Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

ideal_cities = data.loc[(data["Wind Speed"] < 10) & (data["Cloudiness"] == 0) & (data["Max Temp"] > 70) & (data["Max Temp"] < 80)].dropna()

ideal_cities.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
79,Abu Samrah,35.30,37.18,70.74,43,0,5.44,SY,06/21/20
89,Korla,41.76,86.15,71.01,87,0,7.67,CN,06/21/20
126,Aksu,41.12,80.26,70.56,23,0,2.53,CN,06/21/20
196,Kumluca,36.37,30.29,72.68,50,0,4.70,TR,06/21/20
234,Guarapari,-20.67,-40.50,71.01,88,0,3.36,BR,06/21/20


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = ideal_cities.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
79,Abu Samrah,SY,35.30,37.18,
89,Korla,CN,41.76,86.15,
126,Aksu,CN,41.12,80.26,
196,Kumluca,TR,36.37,30.29,
234,Guarapari,BR,-20.67,-40.50,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    params["location"] = f"{lat},{lng}"
    print(f"Getting data for {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except:
        print("Missing info skipping.")
    print("------------")
    time.sleep(2)
print("-------End of Search-------")

Getting data for 79: Abu Samrah.
Missing info skipping.
------------
Getting data for 89: Korla.
Closest hotel in Korla is Kangcheng Jian Guo International Hotel.
------------
Getting data for 126: Aksu.
Closest hotel in Aksu is Pudong Holiday Hotel.
------------
Getting data for 196: Kumluca.
Closest hotel in Kumluca is TurkuazKöy.
------------
Getting data for 234: Guarapari.
Closest hotel in Guarapari is Bristol Guarapari Residence Hotel.
------------
Getting data for 243: Mackay.
Closest hotel in Mackay is Ocean International Hotel.
------------
Getting data for 351: Soyo.
Closest hotel in Soyo is Kinwica.
------------
Getting data for 370: Bāfq.
Closest hotel in Bāfq is Alamdar Hotel.
------------
Getting data for 390: Shache.
Closest hotel in Shache is New Century Hotel.
------------
Getting data for 407: Kutum.
Missing info skipping.
------------
Getting data for 437: Vila Velha.
Closest hotel in Vila Velha is Quality Suites Vila Velha.
------------
Getting data for 452: Cruz Al

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))